# A Simple Generative Adversarial Network with Keras

In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm  #모델이 각 epoch 마다 얼마나 멋진 결과를 내고 있는지 보여줌 

In [6]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist 
from keras.optimizers import Adam
from keras import initializers 

In [8]:
#keras가 Tensorflow 를 백엔드로 사용할 수 있도록 설정합니다.
os.environ["KERAS_BACKEND"]="tensorflow"

#실험을 재현하고 동일한 결과를 얻을 수 있는지 확인하기 위해 seed를 설정합니다. 
np.random.seed(10)

#우리의 랜덤 노이즈 백터의 차원을 설정합니다.
random_dim=100 

## 0~ 9 까지의 단일 자릿수의 이미지 데이터셋 MNIST 사용 

In [41]:
def load_mnist_data():
    #데이터를 로드합니다.
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    #데이터를 -1~ 1 사이 값으로 normalization 합니다. 
    x_train=(x_train.astype(np.float32)-127.5)/127.5  # 0제외한 평균 
    
    #x_train 의 shape를 (6000,28,28) 에서 (60000,784) 로 바꿉니다 
    # 28*28 이 하나의 이미지이므로 한 행으로 변환시킨다. 
    #따라서 우리는 한 row 당 784 columns을 가지게 됩니다. 
    
    x_train=x_train.reshape(60000,784) #차원축소 
    return (x_train,y_train,x_test,y_test)

In [42]:
load_mnist_data()

(array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0

이제 Generator 및 Discriminator 네트워크를 만들어 볼 수 있습니다. 두 네트워크에 모두 Adam Optimizer를 사용합니다.
Generator와 Discriminator의 경우 모두 세 개의 숨겨진 레이어가 있는 신경 네트워크를 생성하며
activation function(활성함수)은 Leaky Relu를 사용합니다. 
또한 Discriminator가 보이지 않는 영상에서 견고성을 향상시킬 수 있도록 Dropout 레이어를 추가해야합니다. 

In [ ]:
#Adam Optimizer를 사용합니다.
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)